In [1]:
import datetime
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import time
from bs4 import BeautifulSoup
import requests
import re
from datetime import datetime
import string
import pytz
from matplotlib import pyplot
import matplotlib.cm as cm

In [2]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

# fetch company names and price

In [19]:
def get_nyse_stocks():
    
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:63.0) Gecko/20100101 Firefox/63.0'}

    list_stock = []
    for letter in string.ascii_uppercase:
        print("Fetching for letter: " + letter)
        
        r = requests.get("https://eoddata.com/stocklist/NYSE/" + letter + ".htm", headers=headers)
        html_doc = r.text
        soup = BeautifulSoup(html_doc, 'html.parser')
        soup_tag = soup.find_all("a", href=True)

        for link in soup_tag:
            text = link.get('href')
            if "/stockquote/NYSE/" in text:
                text = text.split("/")[-1].split(".")[0]

                if text not in list_stock:
                    list_stock.append(text)
                    
    return list_stock

In [213]:
def get_interesting_stocks_wsj(list_stock):
    
    interesting_stocks = []
    
    for symbol in list_stock:
        
        print(symbol)

        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:63.0) Gecko/20100101 Firefox/63.0'}
        r = requests.get("https://www.wsj.com/market-data/quotes/" + symbol + "/research-ratings", headers=headers)
        html_doc = r.text
        soup = BeautifulSoup(html_doc, 'html.parser')
        soup_tag = soup.find_all("span", attrs={"class": "data_data"})

        def get_prices(soup_tag):
            
            current_price_tag = re.findall(r'\d+', str(soup_tag[-1]))
            low_price_tag = re.findall(r'\d+', str(soup_tag[-3]))
            
            def convert_tag_to_price(tag_price):
                
                # price of the action is in thousands (seperated by comma)
                if len(tag_price) == 3:
                    price = int(tag_price[0])*1000+int(tag_price[1])+float(tag_price[2])/100
                else:
                    price = int(tag_price[0])+float(tag_price[1])/100

                return price
            
            current_price = convert_tag_to_price(current_price_tag)
            low_price = convert_tag_to_price(low_price_tag)
            
            return current_price, low_price
        
        def get_number_of_analysts(soup_tag):
            
            analyst_list = [int(re.findall(r'\d+', str(tag))[0]) for tag in soup_tag if len(re.findall(r'\d+', str(tag))) == 1]
            num_categories = int(len(analyst_list)/3)
            num_analysts = np.sum(np.array(analyst_list)[[(index-1)+2*index for index in range(1, num_categories+1)]])
            
            return num_analysts
        
        try:
            current_price, low_price = get_prices(soup_tag)
            number_of_analysts = get_number_of_analysts(soup_tag)
            
            print(current_price, number_of_analysts)
            
            if current_price < low_price and number_of_analysts >= 4:
                interesting_stocks.append((symbol, ((low_price-current_price)/current_price)*100))
                
        except IndexError:
            print("No data found for symbol: " + symbol)
            
    return interesting_stocks

In [269]:
def get_interesting_stocks_zacks(interesting_stocks):
    
    updated_interesting_stocks = []
    
    for entry in interesting_stocks:
        symbol, potential_upside = entry[0], entry[1]
    
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:63.0) Gecko/20100101 Firefox/63.0'}
        r = requests.get("https://www.zacks.com/stock/quote/" + symbol + "?q=aapl", headers=headers)
        html_doc = r.text
        soup = BeautifulSoup(html_doc, 'html.parser')
        soup_tag = soup.find_all("span", attrs={"class": "composite_val"})

        try:
            if len(soup_tag) == 16:
                score_list = [char for tag in soup_tag[4:7] for char in str(tag) if char.isupper()]
                
                if score_list:
                    score_list.sort()

                    if score_list[-1] <= 'B':
                        print(entry, score_list)
                        updated_interesting_stocks.append((symbol, potential_upside, score_list))
                
        except IndexError:
            print("No data found for symbol: " + symbol)
            
    return updated_interesting_stocks
            

In [250]:
# interesting_stocks = get_interesting_stocks_wsj(list_stock)

A
131.64 18
AA
31.57 12
AAC
No data found for symbol: AAC
AAI-B
No data found for symbol: AAI-B
AAI-C
No data found for symbol: AAI-C
AAIC
3.98 2
AAN
24.77 10
AAP
186.38 23
AAQ
No data found for symbol: AAQ
AAT
33.0 5
AB
40.11 8
ABB
31.72 32
ABBV
107.74 23
ABC
116.69 18
ABEV
2.76 13
ABG
198.37 9
ABM
53.23 5
ABR
16.18 4
ABR-A
No data found for symbol: ABR-A
ABR-B
No data found for symbol: ABR-B
ABR-C
No data found for symbol: ABR-C
ABT
120.93 22
AC
No data found for symbol: AC
ACA
62.89 5
ACB
8.3 15
ACC
43.19 10
ACCO
8.42 5
ACEL
11.53 6
ACH
11.26 2
ACI
19.04 18
ACIC
No data found for symbol: ACIC
ACII
No data found for symbol: ACII
ACM
66.76 11
ACN
286.11 28
ACND
No data found for symbol: ACND
ACP
No data found for symbol: ACP
ACR
15.36 2
ACR-C
No data found for symbol: ACR-C
ACRE
13.88 6
ACV
No data found for symbol: ACV
ADC
67.74 13
ADCT
23.24 4
ADE
No data found for symbol: ADE
ADEX
No data found for symbol: ADEX
ADM
58.82 13
ADNT
43.08 12
ADS
110.15 20
ADT
9.46 7
ADX
No data found f

No data found for symbol: BFL
BFLY
No data found for symbol: BFLY
BFS
40.88 3
BFS-D
No data found for symbol: BFS-D
BFS-E
No data found for symbol: BFS-E
BFT
No data found for symbol: BFT
BFY
No data found for symbol: BFY
BFZ
No data found for symbol: BFZ
BG
79.46 9
BGB
No data found for symbol: BGB
BGH
No data found for symbol: BGH
BGIO
No data found for symbol: BGIO
BGR
No data found for symbol: BGR
BGS
29.75 10
BGSF
14.31 2
BGT
No data found for symbol: BGT
BGX
No data found for symbol: BGX
BGY
No data found for symbol: BGY
BH
136.45 0
BHC
29.83 18
BHE
30.9 3
BHK
No data found for symbol: BHK
BHLB
22.33 3
BHP
70.11 16
BHR
6.15 4
BHR-B
No data found for symbol: BHR-B
BHR-D
No data found for symbol: BHR-D
BHV
No data found for symbol: BHV
BHVN
68.55 12
BIF
No data found for symbol: BIF
BIG
67.35 10
BIGZ
No data found for symbol: BIGZ
BILL
151.4 15
BIO
602.05 5
BIP
54.82 12
BIP-A
No data found for symbol: BIP-A
BIP-B
No data found for symbol: BIP-B
BIPC
74.39 1
BIT
No data found for sy

26.16 0.0
CTK
2.32 1
CTLT
108.32 13
CTO
52.87 3
CTOS
9.78 4
CTR
No data found for symbol: CTR
CTS
30.77 3
CTT
10.59 6
CTVA
47.13 21
CUB
74.77 5
CUB-C
No data found for symbol: CUB-C
CUB-D
No data found for symbol: CUB-D
CUB-E
No data found for symbol: CUB-E
CUB-F
No data found for symbol: CUB-F
CUBB
25.97 7
CUBE
39.58 11
CUBI
33.4 8
CUK
23.81 19
CULP
14.85 1
CURO
13.97 5
CUZ
35.41 5
CVA
13.89 7
CVE
7.65 19
CVEO
16.24 1
CVI
18.7 8
CVII
No data found for symbol: CVII
CVNA
259.99 24
CVS
74.77 27
CVX
102.92 29
CW
124.27 5
CWE
No data found for symbol: CWE
CWEN
28.69 9
CWH
38.85 10
CWK
16.67 8
CWT
57.9 6
CX
7.05 22
CXE
No data found for symbol: CXE
CXH
No data found for symbol: CXH
CXP
17.99 4
CXW
7.66 1
CYD
15.77 2
CYH
11.81 10
D
77.0 17
DAC
53.71 4
DAL
48.35 21
DAN
25.91 10
DAO
24.52 11
DAR
73.24 12
DASH
144.73 20
DAVA
81.77 9
DB
12.34 21
DBD
13.61 4
DBI
17.45 6
DBL
No data found for symbol: DBL
DCF
No data found for symbol: DCF
DCI
59.76 7
DCO
64.09 0
DCP
21.9 14
DCP-B
No data found for 

6.83 4
FIS
149.06 38
FIV
No data found for symbol: FIV
FIX
79.39 4
FL
58.28 21
FLC
No data found for symbol: FLC
FLM
No data found for symbol: FLM
FLNG
8.87 8
FLO
24.01 7
FLOW
65.11 9
FLR
22.65 9
FLS
39.62 13
FLT
286.68 21
FLY
16.84 3
FMA
No data found for symbol: FMA
FMAC
No data found for symbol: FMAC
FMC
112.4 20
FMN
No data found for symbol: FMN
FMO
No data found for symbol: FMO
FMS
37.92 18
FMX
80.88 19
FMY
No data found for symbol: FMY
FN
89.17 5
FNB
13.04 8
FNB-E
No data found for symbol: FNB-E
FND
108.91 24
FNF
43.86 5
FNV
134.33 10
FOA
No data found for symbol: FOA
FOE
16.96 6
FOF
No data found for symbol: FOF
FOR
24.79 5
FOUR
97.33 12
FPA
No data found for symbol: FPA
FPAC
No data found for symbol: FPAC
FPF
No data found for symbol: FPF
FPH
7.47 2
FPI
11.45 3
FPI-B
No data found for symbol: FPI-B
FPL
No data found for symbol: FPL
FR
46.9 12
FRA
No data found for symbol: FRA
FRC
173.19 22
FRC-G
No data found for symbol: FRC-G
FRC-H
No data found for symbol: FRC-H
FRC-I
No data

49.48 19
IFN
No data found for symbol: IFN
IFS
30.13 8
IGA
No data found for symbol: IGA
IGD
No data found for symbol: IGD
IGI
No data found for symbol: IGI
IGR
No data found for symbol: IGR
IGT
16.23 8
IH
12.01 2
IHC
41.5 0
IHD
No data found for symbol: IHD
IHG
70.46 22
IHIT
No data found for symbol: IHIT
IHTA
No data found for symbol: IHTA
IIA
No data found for symbol: IIA
IIAC
9.98 0.0
IID
No data found for symbol: IID
IIF
No data found for symbol: IIF
IIIN
31.65 2
IIM
No data found for symbol: IIM
IIP-A
No data found for symbol: IIP-A
IIPR
186.67 7
IMAX
21.53 10
IMP
No data found for symbol: IMP
IMPX
No data found for symbol: IMPX
INFO
101.65 11
INFY
19.06 50
ING
12.34 22
INGR
91.35 6
INN
10.0 5
INN-D
No data found for symbol: INN-D
INN-E
No data found for symbol: INN-E
INS-A
No data found for symbol: INS-A
INSI
No data found for symbol: INSI
INSP
212.6 13
INSW
18.51 9
INT
33.89 2
INVH
32.23 19
IO
2.5300000000000002 3
IP
55.74 16
IPG
29.48 14
IPI
31.99 5
IPO
No data found for symbo

31.69 20
MOT
No data found for symbol: MOT
MOTV
No data found for symbol: MOTV
MOV
29.25 1
MP
35.3 5
MPA
No data found for symbol: MPA
MPC
53.07 16
MPL
No data found for symbol: MPL
MPLN
6.33 4
MPLX
26.0 18
MPV
No data found for symbol: MPV
MPW
21.5 13
MPX
16.71 1
MQT
No data found for symbol: MQT
MQY
No data found for symbol: MQY
MRC
9.09 7
MRK
76.11 23
MRO
10.89 29
MS
80.34 28
MS-A
No data found for symbol: MS-A
MS-E
No data found for symbol: MS-E
MS-F
No data found for symbol: MS-F
MS-I
No data found for symbol: MS-I
MS-K
No data found for symbol: MS-K
MS-L
No data found for symbol: MS-L
MSA
153.48 5
MSB
31.76 1
MSC
13.49 0
MSCI
447.61 11
MSD
No data found for symbol: MSD
MSGE
89.7 9
MSGN
15.83 4
MSGS
182.23 10
MSI
191.06 12
MSM
88.89 12
MSP
24.04 10
MT
29.14 20
MTB
155.59 20
MTCN
68.11 0.0
MTD
1228.78 13
MTDR
23.54 15
MTG
14.27 11
MTH
95.38 9
MTL
1.71 1
MTN
301.63 13
MTOR
28.55 5
MTR
No data found for symbol: MTR
MTRN
66.77 1
MTT
No data found for symbol: MTT
MTW
20.64 0.0
MTX
76.4

No data found for symbol: PFD
PFE
36.51 22
PFGC
57.77 11
PFH
25.7 15
PFL
No data found for symbol: PFL
PFN
No data found for symbol: PFN
PFO
No data found for symbol: PFO
PFS
23.11 5
PFSI
59.01 8
PG
135.88 23
PGP
No data found for symbol: PGP
PGR
95.63 20
PGRE
10.37 8
PGTI
26.07 5
PGZ
No data found for symbol: PGZ
PH
321.05 21
PHD
No data found for symbol: PHD
PHG
60.15 19
PHI
25.74 10
PHK
No data found for symbol: PHK
PHM
54.32 13
PHR
53.52 13
PHT
No data found for symbol: PHT
PHX
2.64 1
PIA
No data found for symbol: PIA
PIAI
No data found for symbol: PIAI
PIC
No data found for symbol: PIC
PICC
No data found for symbol: PICC
PII
139.93 17
PIM
No data found for symbol: PIM
PINE
18.02 6
PING
22.76 13
PINS
85.53 29
PIP
No data found for symbol: PIP
PIPP
No data found for symbol: PIPP
PIPR
115.33 3
PJT
66.91 6
PK
21.19 16
PKE
13.8 1
PKG
138.26 14
PKI
132.76 15
PKO
No data found for symbol: PKO
PKX
73.05 24
PLAN
59.78 18
PLD
108.55 18
PLNT
86.8 16
PLOW
46.12 3
PLT
35.56 6
PLTR
23.2 8
PLYM


13.61 14
SITE
180.62 8
SIX
46.55 14
SJI
24.6 10
SJIJ
25.87 10
SJIU
41.92 0
SJIV
53.68 10
SJM
129.88 17
SJR
26.79 10
SJT
4.11 1
SJW
64.52 6
SKL
No data found for symbol: SKL
SKLZ
16.6 5
SKM
28.0 24
SKT
16.29 8
SKX
43.18 13
SKY
45.83 6
SLA
No data found for symbol: SLA
SLAC
No data found for symbol: SLAC
SLB
26.71 26
SLCA
11.59 4
SLF
51.68 16
SLG
71.8 17
SLG-I
No data found for symbol: SLG-I
SLQT
31.39 11
SM
16.74 14
SMAR
64.37 16
SMFG
7.17 11
SMG
249.89 7
SMHI
5.05 1
SMLP
20.84 5
SMM
No data found for symbol: SMM
SMP
42.62 3
SNA
235.26 10
SNAP
61.62 39
SNDR
25.4 15
SNE
No data found for symbol: SNE
SNI
No data found for symbol: SNI
SNN
38.58 14
SNOW
227.17 26
SNP
52.43 4
SNPR
No data found for symbol: SNPR
SNR
6.75 4
SNV
46.6 17
SNV-D
No data found for symbol: SNV-D
SNV-E
No data found for symbol: SNV-E
SNX
118.47 7
SO
63.02 19
SOA
No data found for symbol: SOA
SOAC
No data found for symbol: SOAC
SOGO
8.54 1
SOI
11.68 9
SOJB
25.75 19
SOJC
26.37 18
SOJD
26.71 19
SOJE
25.34 19
SOL
10.0 3


No data found for symbol: VNO-K
VNO-L
No data found for symbol: VNO-L
VNO-M
No data found for symbol: VNO-M
VNO-N
No data found for symbol: VNO-N
VNT
30.68 11
VNTR
4.6 12
VOC
3.44 1
VOY-B
No data found for symbol: VOY-B
VOYA
66.87 14
VPC
No data found for symbol: VPC
VPG
31.16 3
VPV
No data found for symbol: VPV
VRS
15.37 2
VRT
21.43 10
VRTV
42.86 1
VSH
25.3 8
VST
17.43 12
VSTO
31.96 10
VTA
No data found for symbol: VTA
VTN
No data found for symbol: VTN
VTOL
25.81 1
VTR
54.39 21
VVI
40.79 3
VVNT
13.76 5
VVR
No data found for symbol: VVR
VVV
26.96 9
VYG
No data found for symbol: VYG
VYGG
No data found for symbol: VYGG
VZ
57.57 28
VZIO
No data found for symbol: VZIO
W
320.36 31
WAB
80.13 12
WAL
93.77 12
WALA
25.75 12
WAR
No data found for symbol: WAR
WAT
307.03 16
WBAI
18.8 1
WBK
19.28 1
WBS
57.28 12
WBS-F
No data found for symbol: WBS-F
WBT
15.97 10
WCC
88.92 12
WCC-A
No data found for symbol: WCC-A
WCN
114.27 16
WD
107.4 3
WDR
24.94 5
WEA
No data found for symbol: WEA
WEC
93.25 15
WEI


In [270]:
interesting_stocks = get_interesting_stocks_zacks(interesting_stocks)

('AMX', 2185.714285714286) ['A', 'B', 'B']
('AXTA', 3.5058430717863134) ['A', 'A', 'B']
('BABA', 568.5481536616632) ['A', 'A', 'B']
('BEST', 520.3821656050957) ['A', 'A', 'A']
('CNC', 3.0600919613128266) ['A', 'A', 'B']
('CVA', 0.7919366450683905) ['B', 'B', 'B']
('DBD', 32.2556943423953) ['A', 'B', 'B']
('DCI', 2.0749665327978613) ['B', 'B', 'B']
('DOOR', 3.2565528196981686) ['A', 'A', 'B']
('EBS', 13.501385772733276) ['A', 'A', 'B']
('ELP', 13.513513513513503) ['B', 'B', 'B']
('EVR', 0.8416104329365485) ['A', 'A', 'B']
('GCO', 1.9783805833163346) ['A', 'B', 'B']
('HIG', 0.9796533534287953) ['A', 'A', 'B']
('HUM', 4.456157163392435) ['A', 'B', 'B']
('HUYA', 526.5432098765433) ['A', 'B', 'B']
('KOF', 2039.979231568017) ['A', 'B', 'B']
('LOMA', 5.348460291734199) ['A', 'A', 'A']
('LXFR', 11.867704280155648) ['A', 'A', 'B']
('NCR', 0.9336361342417295) ['A', 'A', 'B']
('NUS', 1.2609858616736656) ['A', 'A', 'B']
('PJT', 15.079958152742496) ['A', 'A', 'B']
('PRG', 29.958960328317374) ['A', 

In [273]:
interesting_stocks = sorted(interesting_stocks, key=lambda tup: tup[1])

In [274]:
interesting_stocks

[('WDR', 0.24057738572573667, ['A', 'B', 'B']),
 ('CVA', 0.7919366450683905, ['B', 'B', 'B']),
 ('EVR', 0.8416104329365485, ['A', 'A', 'B']),
 ('NCR', 0.9336361342417295, ['A', 'A', 'B']),
 ('HIG', 0.9796533534287953, ['A', 'A', 'B']),
 ('NUS', 1.2609858616736656, ['A', 'A', 'B']),
 ('TRGP', 1.7164653528289866, ['A', 'A', 'B']),
 ('GCO', 1.9783805833163346, ['A', 'B', 'B']),
 ('DCI', 2.0749665327978613, ['B', 'B', 'B']),
 ('UNF', 2.6281736647182123, ['A', 'B', 'B']),
 ('RIO', 2.6485869978456513, ['A', 'A', 'B']),
 ('CNC', 3.0600919613128266, ['A', 'A', 'B']),
 ('DOOR', 3.2565528196981686, ['A', 'A', 'B']),
 ('AXTA', 3.5058430717863134, ['A', 'A', 'B']),
 ('HUM', 4.456157163392435, ['A', 'B', 'B']),
 ('TKR', 4.594695554725436, ['B', 'B', 'B']),
 ('LOMA', 5.348460291734199, ['A', 'A', 'A']),
 ('VNT', 7.561929595827902, ['A', 'A', 'A']),
 ('TISI', 8.786610878661094, ['A', 'B', 'B']),
 ('VSTO', 9.511889862327907, ['A', 'A', 'B']),
 ('LXFR', 11.867704280155648, ['A', 'A', 'B']),
 ('EBS', 13